In [343]:
import numpy as np
from collections import OrderedDict as odict
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [367]:
#(hardware name, number of nodes)
filesD = {
         'knl_mpi1':('knl',1), 'knl_mpi2':('knl',2), 'knl_mpi4':('knl',4),
         'skl_mpi1':('skl',1), 'skl_mpi2':('skl',2), 'skl_mpi4':('skl',4),
         'i5':('i5',1),
         'p100_mpi1':('p100',1), 'p100_mpi2':('p100',2), 'p100_mpi4':('p100',4),
         'v100_mpi1':('v100',1), 'v100_mpi2':('v100',2), 'v100_mpi4':('v100',4),
         'gtx1060':('gtx1060',1)
                      }

files = odict(sorted(filesD.items(), key= lambda t : t[1][1]))

In [368]:
pd.set_option('precision',1)
names={'axpby':3,'dot':2}
values = []
for f, v in files.items() :#{'knl_mpi2':('knl',2)}.items():#files.items():  
    df=pd.read_csv('benchmark_'+f+'.csv', delimiter=' ')
    #add size and get rid of non-relevant columns
    df.insert(0,'size', 8*df['n']*df['n']*df['Nx']*df['Ny']/1e6/v[1]) #inplace transformation
    for name,memops in names.items() :
        df.insert(0,name+'_bw',df['size']/1000*memops/df[name])
    agg=df.groupby(['n', 'Nx','Ny','size']).agg(['mean', 'std'])
    agg=agg.reset_index(level=['n','Nx','Ny','size'])
    agg.sort_values(by='size',inplace=True) #sort by size
    df.sort_values(by='size',inplace=True)
    
    n =3
    s =3
    if f!='i5' :
        s = s*10 #here we assume that REPETITIONS is always 10
    line = []
    l=len(df)
    line.append(v[0])
    line.append(v[1])
    #line.append(df[0:n]['axpby'].mean()/1e-6)
    #line.append(df[0:n]['axpby'].min()/1e-6)
    #line.append(df[l-n:l]['axpby_bw'].mean())
    #line.append(df[l-n:l]['axpby_bw'].std())
    #line.append(df[0:n]['dot'].mean()/1e-6)
    #line.append(df[0:n]['dot'].min()/1e-6)
    #line.append(df[l-n:l]['dot_bw'].mean())
    #line.append(df[l-n:l]['dot_bw'].std())
    line.append(agg[0:n][('axpby','mean')].mean()/1e-6)
    line.append(agg[0:n][('axpby','mean')].min()/1e-6)
    line.append(df[l-s:l]['axpby_bw'].mean())
    line.append(df[l-s:l]['axpby_bw'].std())
    line.append(agg[0:n][('dot','mean')].mean()/1e-6)
    line.append(agg[0:n][('dot','mean')].min()/1e-6)
    line.append(df[l-s:l]['dot_bw'].mean())
    line.append(df[l-s:l]['dot_bw'].std())
    values.append(line)
                
cols=pd.MultiIndex.from_tuples([('arch','',''),('nodes','',''),
      ('axpby','lat','avg'),('axpby','lat','min'),
      ('axpby','bw','avg'),('axpby','bw','std'),
      ('dot','lat','avg'),('dot','lat','min'),
      ('dot','bw','avg'),('dot','bw','std'),
     ])
arr = pd.DataFrame(values,index=files.keys(), columns=cols)
arr.sort_values(by='arch',inplace=True)
arr.set_index(['arch','nodes'],inplace=True)
#arr.loc[:,[('dot','bw','avg'),('dot','lat','avg')]]
#arr

In [369]:
#arr=arr.reset_index()

In [370]:
def toString(x): 
    string = '%.1f'% x
    return string
for n in ['axpby','dot']:
    arr.loc[:,(n,'bw','string')]= arr[n]['bw']['avg'].apply(toString) +" ± "+arr[n]['bw']['std'].apply(toString)

nicetable=arr[[('axpby','lat','min'),('axpby','bw','string'),('dot','lat','min'),('dot','bw','string')]]
drop = nicetable.columns.droplevel(2)
nicetable.columns=drop
nicetable

axpby                  dot              
                lat            bw    lat            bw
arch    nodes                                         
gtx1060 1       3.5   157.0 ± 0.1  131.6    26.5 ± 0.1
i5      1      11.6    29.6 ± 0.0  117.3     9.3 ± 0.1
knl     1      14.4  447.0 ± 14.8   67.3   143.3 ± 2.2
        2      14.9   453.4 ± 8.9   93.9   128.9 ± 8.2
        4      13.8  450.0 ± 26.2  128.5   112.9 ± 5.8
p100    1       3.1   552.8 ± 1.0   56.4   345.8 ± 1.9
        2       3.3   554.2 ± 0.1   56.1   338.2 ± 3.2
        4       3.1   554.8 ± 1.7   68.0   320.3 ± 7.5
skl     1       6.1   206.8 ± 3.0   24.5   193.6 ± 8.6
        2       6.9   210.4 ± 6.1   32.6   184.5 ± 8.4
        4       6.0  225.5 ± 12.6   40.7  173.0 ± 12.3
v100    1       3.1   848.3 ± 0.7   41.5   591.6 ± 2.9
        2       3.1   847.9 ± 0.4   48.2   575.3 ± 7.0
        4       3.4   844.9 ± 1.8   59.2  525.7 ± 13.9